<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

這個檔案是拿來抓取球員在不同進壘位置的揮棒或不揮棒之類的資料

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import copy
import pickle
import os 

In [2]:
csv_path = 'MLB_Player_Stats_CSV\\'
player_type = 'batter'
years = [*range(2017,2024)]

樓下這個是先跑過一次save_swing_take_data_from_old_file之後，算出的每年全聯盟的 swing take 比例

In [3]:
swing_take_propotion = {
2017: [0.191, 0.223, 0.051, 0.004, 0.071, 0.203, 0.177, 0.080],
2018: [0.190, 0.224, 0.051, 0.005, 0.071, 0.203, 0.174, 0.082],
2019: [0.185, 0.228, 0.055, 0.006, 0.067, 0.196, 0.178, 0.085],
2020: [0.184, 0.222, 0.053, 0.005, 0.070, 0.200, 0.178, 0.088],
2021: [0.195, 0.225, 0.050, 0.005, 0.069, 0.198, 0.174, 0.085],
2022: [0.190, 0.230, 0.054, 0.005, 0.070, 0.198, 0.172, 0.080],
2023: [0.192, 0.226, 0.052, 0.005, 0.072, 0.200, 0.172, 0.080]
}

抓單一球員單一球季的資料

In [8]:
class Swing_Take():
    def __init__(self, player_type, all_swing_take=None, swing_take_propotion=swing_take_propotion):
        if all_swing_take is None:
            self.all_swing_take = {}
        else:
            self.all_swing_take = all_swing_take
        self.player_type = player_type
        self.swing_take_propotion = swing_take_propotion
        self.hand_put_in = []
        
    def add_new_data(self, year, player_id, swing_take):
        if not str(year) in self.all_swing_take.keys():
            self.all_swing_take[str(year)] = {'player_id':[player_id], 'swing_take':[swing_take]}
        else:
            self.all_swing_take[str(year)]['player_id'].append(player_id)
            self.all_swing_take[str(year)]['swing_take'].append(swing_take)
    
    def grab_detailed_swing_take_stat(self, player_id, year, pitch_count):
        if self.player_type == 'batter':
            url = f'https://baseballsavant.mlb.com/visuals/swing-take?playerId={player_id}&playerSet=hitters&year={year}'
        elif self.player_type == 'pitcher':
            url = f'https://baseballsavant.mlb.com/visuals/swing-take?playerId={player_id}&playerSet=pitchers&year={year}'
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)
        try:
            driver.get(url)
            driver.implicitly_wait(20)
            soup = bs(driver.page_source.encode('utf-8'), 'html.parser')
            table = soup.find('div',{'class':'vis-grid'})
        except:
            driver.get(url)
            driver.implicitly_wait(20)
            soup = bs(driver.page_source.encode('utf-8'), 'html.parser')
            table = soup.find('div',{'class':'vis-grid'})
        
#         print(str(soup.find('div',{'class':'leaderboards'}).find('a')))
#         id_we_got = int(str(soup.find('div',{'class':'leaderboards'}).find('a'))\
#                         .split('swing-take_playerId=')[1].split('&')[0])
        
#         if not id_we_got == player_id:
#             print('Got wrong player. The player we requested might be a pitcher.')
#             swing_take = [int(i * pitch_count) for i in self.swing_take_propotion[year]]
#             self.hand_put_in.append([year, player_id, pitch_count])
#             self.add_new_data(year, player_id, swing_take)
#             return
        if not table:
            print('No data for this player')
            swing_take = [int(i * pitch_count) for i in self.swing_take_propotion[year]]
            self.hand_put_in.append([year, player_id, pitch_count])
            self.add_new_data(year, player_id, swing_take)
            return
        swing_html_list = table.findAll('text',{'class':'player-swing-count'})
        swing_list = []
        for this_html in swing_html_list:
            swing_list.append(int(this_html.get_text()))

        take_html_list = table.findAll('text',{'class':'player-take-count'})
        take_list = []
        for this_html in take_html_list:
            take_list.append(int(this_html.get_text()))

        swing_take = swing_list + take_list
        
        if sum(swing_take) > pitch_count * 1.2:
            print('Got impossible numbers. The player we requested might be a pitcher.')
            swing_take = [int(i * pitch_count) for i in self.swing_take_propotion[year]]
            self.hand_put_in.append([year, player_id, pitch_count])
        print(swing_take)
        self.add_new_data(year, player_id, swing_take)
        
    
    def grab_data_single_year(self, year, player_idx, player_pitch_count_list):
        t0 = time.time()
        for i, ID in enumerate(player_idx):
            print(f'{year}:{ID}')
            if str(year) in self.all_swing_take.keys():
                if ID in self.all_swing_take[str(year)]['player_id']:
                    continue
            self.grab_detailed_swing_take_stat(ID, year, player_pitch_count_list[i])
#             try:
#                 self.grab_detailed_swing_take_stat(ID, year, player_pitch_count_list[i])
#             except:
#                 print('There might be no this player on this page.')
            t1 = time.time()
            print(f'{i+1}/{len(player_idx)}, process time: {(t1-t0)/60:.2f} min')
        

In [9]:
if os.path.exists(f'all_swing_take_{player_type}.pickle'):
    with open(f'all_swing_take_{player_type}.pickle', 'rb') as f:
        all_swing_take = pickle.load(f)
    swing_take = Swing_Take(player_type, all_swing_take=all_swing_take)
else:
    swing_take = Swing_Take(player_type)

In [12]:
old_dfs = {}
for year in years:
    filename = f'{csv_path}{year}_{player_type}_arranged_V2.csv'
    old_dfs[year] = pd.read_csv(filename)[['player_id', 'pitch_count', 'pa']]
    swing_take.grab_data_single_year(year, old_dfs[year]['player_id'],
                                     old_dfs[year]['pitch_count'])

2017:112526
2017:134181
2017:136860
2017:150029
2017:276520
2017:282332
2017:285079
2017:400085
2017:400121
2017:400284
2017:405395
2017:407793
2017:407812
2017:407822
2017:408045
2017:408234
2017:408236
2017:408252
2017:408314
2017:425509
2017:425772
2017:425783
2017:425784
2017:425794
2017:425844
2017:425877
2017:429664
2017:429665
2017:429666
2017:429711
2017:429719
2017:429722
2017:430001
2017:430599
2017:430605
2017:430661
2017:430832
2017:430912
2017:430935
2017:430945
2017:430947
2017:431094
2017:431145
2017:433217
2017:434158
2017:434378
2017:434563
2017:434567
2017:434604
2017:434622
2017:434628
2017:434658
2017:434670
2017:434671
2017:434672
2017:434778
2017:435062
2017:435063
2017:435079
2017:435221
2017:435263
2017:435522
2017:435559
2017:435622
2017:443558
2017:444379
2017:444432
2017:444468
2017:444482
2017:444489
2017:444843
2017:444857
2017:444876
2017:445055
2017:445060
2017:445276
2017:445926
2017:445988
2017:446263
2017:446308
2017:446334
2017:446359
2017:446372
2017

No data for this player
494/987, process time: 0.16 min
2019:596059
2019:596103
2019:596105
2019:596112
2019:596115
2019:596117
2019:596119
2019:596129
2019:596133
2019:596142
2019:596144
2019:596146
2019:596451
2019:596748
2019:596825
2019:596847
2019:598264
No data for this player
511/987, process time: 0.28 min
2019:598265
2019:598271
Got impossible numbers. The player we requested might be a pitcher.
[0, 1, 0, 0, 0, 0, 0, 0]
513/987, process time: 0.40 min
2019:598284
2019:598286
Got impossible numbers. The player we requested might be a pitcher.
[0, 1, 0, 0, 0, 0, 0, 0]
515/987, process time: 0.52 min
2019:600303
2019:600466
2019:600474
2019:600524
2019:600858
2019:600869
2019:600968
Got impossible numbers. The player we requested might be a pitcher.
[0, 0, 0, 0, 0, 0, 0, 0]
522/987, process time: 0.64 min
2019:601713
2019:602074
2019:602922
2019:605113
2019:605119
2019:605131
2019:605135
Got impossible numbers. The player we requested might be a pitcher.
[2, 3, 0, 0, 1, 3, 2, 1]


Got impossible numbers. The player we requested might be a pitcher.
[3, 3, 0, 0, 1, 3, 3, 1]
713/987, process time: 6.00 min
2019:623520
2019:623912
2019:623993
2019:624133
2019:624407
2019:624413
2019:624414
2019:624415
2019:624419
2019:624424
2019:624428
2019:624431
2019:624512
2019:624513
2019:624577
2019:624585
2019:624641
2019:625510
2019:625643
Got impossible numbers. The player we requested might be a pitcher.
[2, 3, 0, 0, 1, 3, 2, 1]
732/987, process time: 6.13 min
2019:628317
2019:628338
2019:628356
2019:628452
Got impossible numbers. The player we requested might be a pitcher.
[1, 1, 0, 0, 0, 1, 1, 0]
736/987, process time: 6.26 min
2019:628711
2019:630023
Got impossible numbers. The player we requested might be a pitcher.
[1, 1, 0, 0, 0, 1, 1, 0]
738/987, process time: 6.38 min
2019:630111
2019:640447
2019:640449
2019:640457
2019:640458
2019:640460
2019:640461
2019:640470
2019:641149
Got impossible numbers. The player we requested might be a pitcher.
[2, 2, 0, 0, 0, 2, 2, 1]

909/987, process time: 11.70 min
2019:663531
Got impossible numbers. The player we requested might be a pitcher.
[2, 3, 0, 0, 1, 2, 2, 1]
910/987, process time: 11.84 min
2019:663538
2019:663567
2019:663586
2019:663656
2019:663757
2019:663776
No data for this player
916/987, process time: 11.95 min
2019:663898
2019:663978
2019:663993
2019:664023
2019:664034
2019:664040
2019:664041
2019:664045
No data for this player
924/987, process time: 12.07 min
2019:664056
2019:664057
2019:664058
2019:664059
2019:664062
2019:664068
2019:664119
2019:664192
2019:664196
Got impossible numbers. The player we requested might be a pitcher.
[1, 1, 0, 0, 0, 1, 1, 0]
933/987, process time: 12.19 min
2019:664199
2019:664208
Got impossible numbers. The player we requested might be a pitcher.
[2, 3, 0, 0, 0, 2, 2, 1]
935/987, process time: 12.31 min
2019:664238
2019:664247
2019:664353
Got impossible numbers. The player we requested might be a pitcher.
[4, 5, 1, 0, 1, 5, 4, 2]
938/987, process time: 12.43 min
2

Got impossible numbers. The player we requested might be a pitcher.
[0, 0, 0, 0, 0, 0, 0, 0]
481/580, process time: 0.72 min
2020:657656
2020:658069
2020:658668
2020:660162
2020:660271
2020:660294
2020:660620
2020:660670
2020:660688
2020:660731
2020:660821
2020:661388
2020:662139
2020:663330
2020:663411
2020:663538
2020:663586
2020:663611
2020:663624
2020:663647
2020:663656
2020:663662
2020:663698
2020:663757
2020:663886
2020:663898
2020:663905
2020:663993
2020:664023
2020:664029
2020:664034
2020:664040
2020:664041
2020:664056
2020:664057
2020:664058
2020:664059
2020:664068
2020:664119
2020:664238
2020:664247
2020:664314
2020:664334
2020:664702
2020:664761
2020:664774
2020:664901
2020:664913
2020:665120
2020:665487
2020:665489
2020:665506
2020:665561
2020:665742
2020:665750
2020:665862
2020:665926
2020:665947
2020:666158
2020:666160
2020:666176
2020:666182
2020:666185
2020:666198
2020:666915
2020:666931
2020:666969
2020:666971
2020:667670
2020:668227
2020:668800
2020:668804
2020:668942

...

Got impossible numbers. The player we requested might be a pitcher.
[2, 2, 0, 0, 0, 2, 2, 1]
674/1045, process time: 15.46 min
2021:643327
2021:643376
2021:643377
Got impossible numbers. The player we requested might be a pitcher.
[2, 2, 0, 0, 0, 2, 2, 1]
677/1045, process time: 15.58 min
2021:643393
2021:643396
2021:643418
2021:643436
2021:643446
2021:643524
2021:643565
2021:643778
Got impossible numbers. The player we requested might be a pitcher.
[0, 0, 0, 0, 0, 0, 0, 0]
685/1045, process time: 15.70 min
2021:644374
2021:644433
2021:645261
2021:645277
2021:645302
2021:645305
2021:645801
2021:646240
2021:647304
2021:647315
Got impossible numbers. The player we requested might be a pitcher.
[1, 1, 0, 0, 0, 1, 1, 0]
695/1045, process time: 15.82 min
2021:647351
2021:649557
2021:649966
2021:650331
2021:650333
2021:650347
Got impossible numbers. The player we requested might be a pitcher.
[0, 0, 0, 0, 0, 0, 0, 0]
701/1045, process time: 15.94 min
2021:650391
2021:650402
2021:650489
2021:

839/1045, process time: 22.03 min
2021:663757
2021:663765
2021:663776
Got impossible numbers. The player we requested might be a pitcher.
[2, 3, 0, 0, 0, 2, 2, 1]
842/1045, process time: 22.15 min
2021:663799
2021:663837
2021:663845
2021:663853
2021:663886
2021:663897
2021:663898
2021:663905
2021:663978
2021:663993
2021:664011
Got impossible numbers. The player we requested might be a pitcher.
[2, 2, 0, 0, 0, 2, 2, 1]
853/1045, process time: 22.26 min
2021:664023
2021:664034
2021:664040
2021:664041
2021:664045
Got impossible numbers. The player we requested might be a pitcher.
[0, 1, 0, 0, 0, 0, 0, 0]
858/1045, process time: 22.38 min
2021:664056
2021:664057
2021:664058
2021:664059
2021:664062
2021:664068
2021:664074
Got impossible numbers. The player we requested might be a pitcher.
[5, 5, 1, 0, 1, 5, 4, 2]
865/1045, process time: 22.50 min
2021:664076
Got impossible numbers. The player we requested might be a pitcher.
[1, 2, 0, 0, 0, 1, 1, 0]
866/1045, process time: 22.62 min
2021:66

Got impossible numbers. The player we requested might be a pitcher.
[0, 0, 0, 0, 0, 0, 0, 0]
1022/1045, process time: 28.08 min
2021:676477
Got impossible numbers. The player we requested might be a pitcher.
[0, 1, 0, 0, 0, 0, 0, 0]
1023/1045, process time: 28.19 min
2021:676480
2021:676596
Got impossible numbers. The player we requested might be a pitcher.
[0, 0, 0, 0, 0, 0, 0, 0]
1025/1045, process time: 28.33 min
2021:676604
Got impossible numbers. The player we requested might be a pitcher.
[1, 1, 0, 0, 0, 1, 1, 0]
1026/1045, process time: 28.45 min
2021:676646
2021:676694
2021:676701
2021:676801
2021:676840
2021:676879
Got impossible numbers. The player we requested might be a pitcher.
[3, 3, 0, 0, 1, 3, 2, 1]
1032/1045, process time: 28.58 min
2021:676900
Got impossible numbers. The player we requested might be a pitcher.
[4, 5, 1, 0, 1, 4, 4, 2]
1033/1045, process time: 28.70 min
2021:677551
2021:677651
Got impossible numbers. The player we requested might be a pitcher.
[2, 3, 0

Got impossible numbers. The player we requested might be a pitcher.
[3, 3, 0, 0, 1, 3, 2, 1]
488/690, process time: 1.07 min
2022:664850
2022:664874
2022:664901
2022:664913
2022:664983
2022:665019
2022:665120
2022:665154
2022:665155
2022:665161
2022:665482
2022:665489
2022:665506
2022:665650
2022:665665
No data for this player
503/690, process time: 1.18 min
2022:665742
2022:665750
2022:665828
2022:665833
2022:665861
2022:665862
2022:665922
Got impossible numbers. The player we requested might be a pitcher.
[6, 7, 1, 0, 2, 6, 5, 2]
510/690, process time: 1.30 min
2022:665923
2022:665926
2022:666018
2022:666023
Got impossible numbers. The player we requested might be a pitcher.
[4, 5, 1, 0, 1, 4, 4, 1]
514/690, process time: 1.41 min
2022:666128
Got impossible numbers. The player we requested might be a pitcher.
[4, 5, 1, 0, 1, 4, 4, 1]
515/690, process time: 1.53 min
2022:666134
2022:666135
2022:666137
2022:666139
2022:666158
2022:666160
2022:666176
2022:666179
2022:666181
2022:666182


[388, 495, 140, 12, 100, 243, 219, 152]
265/654, process time: 0.46 min
2023:643348
2023:643376
2023:643393
2023:643396
2023:643446
2023:643565
2023:644374
2023:644433
2023:645277
2023:645302
2023:645444
2023:645801
2023:646240
2023:647304
2023:647351
2023:649557
2023:649966
2023:650333
2023:650391
2023:650402
2023:650489
2023:650490
2023:650559
2023:650619
2023:650859
2023:650907
2023:655316
2023:656024
2023:656180
2023:656185
2023:656248
2023:656305
2023:656308
2023:656371
2023:656403
2023:656413
2023:656448
2023:656484
Got impossible numbers. The player we requested might be a pitcher.
[4, 4, 1, 0, 1, 4, 3, 1]
303/654, process time: 0.59 min
2023:656495
2023:656514
2023:656537
2023:656541
2023:656582
2023:656669
2023:656716
2023:656775
2023:656811
2023:656821
2023:656896
2023:656941
2023:656976
2023:657041
2023:657061
2023:657077
2023:657088
2023:657136
2023:657247
2023:657557
2023:657656
2023:657757
2023:658668
2023:660162
2023:660271
2023:660620
2023:660634
2023:660636
2023:660644

In [13]:
i = 100
print(all_swing_take['2017']['player_id'][i])
print(all_swing_take['2017']['swing_take'][i])

453056
[323, 316, 57, 2, 124, 373, 303, 112]


存檔

In [14]:
with open(f'all_swing_take_{player_type}.pickle', 'wb') as f:
    pickle.dump(swing_take.all_swing_take, f)

In [15]:
all_swing_take

{'2017': {'player_id': [112526,
   134181,
   136860,
   150029,
   276520,
   285079,
   400085,
   400121,
   400284,
   405395,
   407793,
   407812,
   408045,
   408234,
   408236,
   408252,
   408314,
   425509,
   425772,
   425783,
   425784,
   425794,
   425844,
   425877,
   429664,
   429665,
   429666,
   429711,
   429719,
   429722,
   430001,
   430605,
   430832,
   430912,
   430935,
   430945,
   430947,
   431094,
   431145,
   433217,
   434158,
   434563,
   434567,
   434604,
   434658,
   434670,
   434671,
   434778,
   435062,
   435063,
   435079,
   435221,
   435263,
   435522,
   435559,
   435622,
   443558,
   444379,
   444432,
   444482,
   444489,
   444843,
   444857,
   444876,
   445055,
   445988,
   446263,
   446308,
   446334,
   446359,
   446381,
   446386,
   446481,
   446653,
   448179,
   448602,
   448605,
   448801,
   448802,
   448855,
   449107,
   449181,
   450172,
   450203,
   450306,
   450308,
   450314,
   451089,
   451192,


檢查隨機一球員的資料

In [16]:
import random
year = 2019
filename = f'{csv_path}{year}_{player_type}_arranged_V2.csv'
df = pd.read_csv(filename).drop('Unnamed: 0', axis=1)
player_list = list(df['player_id'])
ID = random.randint(0, len(player_list))
player_id = player_list[ID]
print(player_id)
print(all_swing_take[str(year)]['swing_take'][all_swing_take[str(year)]['player_id'].index(player_id)])

456078
[177, 206, 46, 3, 90, 216, 177, 76]


In [17]:
for year in years:
    swing_take_propotion = np.sum(np.array(all_swing_take[str(year)]['swing_take']), axis=0)\
                        / np.sum(np.array(all_swing_take[str(year)]['swing_take']))
    print(year)
    print(swing_take_propotion)
    this_swing_take_list = all_swing_take[str(year)]['swing_take']
    this_ST_ID_list = all_swing_take[str(year)]['player_id']
    for i, ID in enumerate(old_dfs[year]['player_id']):
        this_pa = old_dfs[year]['pa'].iloc[i]
        if ID in this_ST_ID_list:
            j = this_ST_ID_list.index(ID)
            this_ST = this_swing_take_list[j]
            this_sum_ST = sum(this_ST)
            print(f'{year}-{ID}: pa={this_pa}, sum_ST={this_sum_ST}')
            print(this_ST)
#         this_pa = old_dfs[year][old_dfs[year]['player_id']==ID]['pa'].iloc[0]
#         if sum(all_swing_take[str(year)]['swing_take'][i]) > 1.1 * this_pa:
#             print(this_pa)
#             print(all_swing_take[str(year)]['swing_take'])

2017
[0.19033547 0.22246247 0.05044498 0.00430388 0.07090732 0.20349761
 0.1773385  0.08070976]
2017-112526: pa=20, sum_ST=61
[12, 14, 3, 0, 4, 12, 11, 5]
2017-134181: pa=389, sum_ST=1573
[273, 350, 105, 7, 112, 301, 299, 126]
2017-136860: pa=509, sum_ST=2013
[331, 488, 128, 16, 129, 392, 364, 165]
2017-150029: pa=289, sum_ST=1400
[238, 242, 47, 8, 141, 365, 256, 103]
2017-276520: pa=29, sum_ST=80
[16, 18, 4, 0, 5, 17, 14, 6]
2017-282332: pa=4, sum_ST=13
[3, 3, 0, 0, 1, 3, 2, 1]
2017-285079: pa=68, sum_ST=212
[46, 59, 44, 9, 13, 18, 16, 7]
2017-400085: pa=215, sum_ST=865
[155, 199, 38, 5, 89, 190, 134, 55]
2017-400121: pa=435, sum_ST=1653
[308, 375, 85, 4, 106, 330, 302, 143]
2017-400284: pa=353, sum_ST=1397
[256, 251, 35, 1, 129, 347, 286, 92]
2017-405395: pa=636, sum_ST=2477
[425, 562, 167, 17, 205, 457, 432, 212]
2017-407793: pa=61, sum_ST=222
[38, 57, 10, 0, 23, 54, 29, 11]
2017-407812: pa=427, sum_ST=1737
[314, 363, 84, 12, 94, 372, 333, 165]
2017-407822: pa=2, sum_ST=0
[0, 0, 0, 

把新資料與 *_arranged_V2.csv 結合，輸出成 *_arranged_V3.csv

In [18]:
for year in years:
    filename = f'{csv_path}{year}_{player_type}_arranged_V2.csv'
    df = pd.read_csv(filename)
    if 'Unnamed: 0' in  df.columns:
        df = df.drop('Unnamed: 0', axis=1)
    
    this_swing_take_dict = all_swing_take[str(year)]
    player_id_list = this_swing_take_dict['player_id']
    this_data = np.array(this_swing_take_dict['swing_take']).T
    swings = this_data[0:4]
    takes = this_data[4:8]
    swing_rates = swings / (swings + takes) * 100
    region_rates = (swings + takes) / sum(swings + takes) * 100
    this_dict = {'player_id': player_id_list,
                'Heart_Swing': list(swings[0]),
                'Shadow_Swing': list(swings[1]),
                'Chase_Swing': list(swings[2]),
                'Waste_Swing': list(swings[3]),
                'Heart_Take': list(takes[0]),
                'Shadow_Take': list(takes[1]),
                'Chase_Take': list(takes[2]),
                'Waste_Take': list(takes[3]),
                'Heart_SwRate': list(swing_rates[0]),
                'Shadow_SwRate': list(swing_rates[1]),
                'Chase_SwRate': list(swing_rates[2]),
                'Waste_SwRate': list(swing_rates[3]),
                'Heart_Rate': list(region_rates[0]),
                'Shadow_Rate': list(region_rates[1]),
                'Chase_Rate': list(region_rates[2]),
                'Waste_Rate': list(region_rates[3])}
    swing_take_df = pd.DataFrame(this_dict)
    print([year, len(df), len(swing_take_df)])
    new_df = pd.merge(df, swing_take_df, on='player_id', how='left')
    output_filename = f'{csv_path}{year}_{player_type}_arranged_V3.csv'
    new_df.to_csv(output_filename)

C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:12: RuntimeWarning: invalid value encountered in divide
  swing_rates = swings / (swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:13: RuntimeWarning: invalid value encountered in divide
  region_rates = (swings + takes) / sum(swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:12: RuntimeWarning: invalid value encountered in divide
  swing_rates = swings / (swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:13: RuntimeWarning: invalid value encountered in divide
  region_rates = (swings + takes) / sum(swings + takes) * 100


[2017, 956, 956]
[2018, 988, 989]


C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:12: RuntimeWarning: invalid value encountered in divide
  swing_rates = swings / (swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:13: RuntimeWarning: invalid value encountered in divide
  region_rates = (swings + takes) / sum(swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:12: RuntimeWarning: invalid value encountered in divide
  swing_rates = swings / (swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:13: RuntimeWarning: invalid value encountered in divide
  region_rates = (swings + takes) / sum(swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:12: RuntimeWarning: invalid value encountered in divide
  swing_rates = swings / (swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:13: RuntimeWarning: invalid value encountered in divide
  region_rat

[2019, 987, 988]
[2020, 580, 581]
[2021, 1045, 1046]
[2022, 690, 691]
[2023, 654, 654]


C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:12: RuntimeWarning: invalid value encountered in divide
  swing_rates = swings / (swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:13: RuntimeWarning: invalid value encountered in divide
  region_rates = (swings + takes) / sum(swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:12: RuntimeWarning: invalid value encountered in divide
  swing_rates = swings / (swings + takes) * 100
C:\Users\skyja\AppData\Local\Temp\ipykernel_65420\3922322754.py:13: RuntimeWarning: invalid value encountered in divide
  region_rates = (swings + takes) / sum(swings + takes) * 100


In [63]:
new_df[np.isnan(new_df['Shadow_Swing'])]

,player_id,player_name,season_id,player_age,pa,pitch_count,in_zone_percent,out_zone_percent,edge_percent,whiff_percent,...,Chase_Take,Waste_Take,Heart_SwRate,Shadow_SwRate,Chase_SwRate,Waste_SwRate,Heart_Rate,Shadow_Rate,Chase_Rate,Waste_Rate
55,543482,"Maggi, Drew",2023,34,6,25,64.0,36.0,40.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,605242,"Fulmer, Michael",2023,30,1,5,80.0,20.0,40.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,623912,"Ramirez, Harold",2023,28,434,1564,45.3,54.7,40.6,28.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,643289,"Dubon, Mauricio",2023,28,492,1671,50.4,49.6,42.2,17.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,656484,"Gray, Tristan",2023,27,5,21,52.4,47.6,52.4,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,663731,"Kaiser, Connor",2023,26,4,15,66.7,33.3,46.7,37.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,669200,"McCoy, Mason",2023,28,1,6,33.3,66.7,33.3,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,669699,"Shewmake, Braden",2023,25,4,9,66.7,33.3,55.6,42.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,676070,"Amaya, Jacob",2023,24,9,28,57.1,42.9,46.4,13.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,676882,"Seagle, Chandler",2023,27,1,1,100.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
